In [1]:
import torch
import numpy as np
import pickle

from HMM_Likelyhood import HMM_log_likelihood,Optimizationloop, InitializeParameters,Accumulated_HHM_LL
with open('.DataPhase.pickle','rb') as f:
    X = pickle.load(f) 
X_tensor = torch.from_numpy(X)
#X_tensor = torch.reshape(X_tensor.T,(-1,90,330))
torch.set_default_dtype(torch.float64)

In [2]:
X.shape

(90, 65670)

In [8]:
# Initialize parameters to tune:

K=5
p=90
Subjects = 199

pi,kappa,mu,Tk = InitializeParameters(Subjects,p,K)

learning_rate = 2
n_iters = 30

torch.autograd.set_detect_anomaly(False)
Parameters = [
    {'params':pi},
    {'params':kappa},
    {'params':mu},
    {'params':Tk},
]

Adam = torch.optim.Adam(Parameters,lr=learning_rate)




In [10]:
X_test = X_tensor[:,0:330*(Subjects+1)]
Pi_fit,Kappa_fit,mu_fit,Tk_fit = Optimizationloop(X_tensor,[pi,kappa,mu,Tk],lose=Accumulated_HHM_LL,Optimizer=Adam,n=Subjects,n_iters=n_iters,K=K)

epoch 1; Log-Likelihood = -28211.06169486732
epoch 2; Log-Likelihood = -28251.752031594242
epoch 3; Log-Likelihood = -28281.845089113463
epoch 4; Log-Likelihood = -28303.27521891221
epoch 5; Log-Likelihood = -28317.75498903338
epoch 6; Log-Likelihood = -28326.46207466197
epoch 7; Log-Likelihood = -28330.51001832236
epoch 8; Log-Likelihood = -28331.23810076582
epoch 9; Log-Likelihood = -28329.9904949842
epoch 10; Log-Likelihood = -28327.67548039011
epoch 11; Log-Likelihood = -28324.614115806668
epoch 12; Log-Likelihood = -28320.948230216778
epoch 13; Log-Likelihood = -28316.942705768346
epoch 14; Log-Likelihood = -28312.870211487178
epoch 15; Log-Likelihood = -28308.954780033797
epoch 16; Log-Likelihood = -28305.41659619994
epoch 17; Log-Likelihood = -28302.429836167234
epoch 18; Log-Likelihood = -28300.052429553925
epoch 19; Log-Likelihood = -28298.238628191884
epoch 20; Log-Likelihood = -28296.917211022028
epoch 21; Log-Likelihood = -28296.042250968763
epoch 22; Log-Likelihood = -2829

In [11]:
Softmax = torch.nn.Softmax(0)
Softplus = torch.nn.Softplus()

pi_est = Softmax(Pi_fit)
kappa_est = Softplus(Kappa_fit)
#kappa_est = Kappa_fit
mu_est = mu /torch.sqrt((mu_fit * mu_fit).sum(axis=0))

In [16]:
print(Softmax(Tk_fit[10]))

tensor([[2.2110e-11, 3.0512e-01, 3.4371e-01, 3.4135e-01, 5.0008e-01],
        [5.0179e-01, 1.2688e-11, 1.4209e-11, 3.2775e-01, 2.1447e-11],
        [2.1206e-11, 3.0302e-01, 1.4362e-11, 1.4876e-11, 2.0977e-11],
        [2.2728e-11, 3.0744e-01, 3.4097e-01, 1.4353e-11, 4.9992e-01],
        [4.9821e-01, 8.4415e-02, 3.1532e-01, 3.3091e-01, 2.0774e-11]],
       grad_fn=<SoftmaxBackward>)


In [13]:
import pandas as pd
labels = pd.read_csv("AALlabels.csv")
label=labels['labels'].values
import plotly.express as px
for model in range(K):
    print(f"Model {model}: Pi = {pi_est[model]} , kappa = {kappa_est[model]}")
    fig = px.bar(y=mu_est[:,model].detach().numpy(),x=label)
    #fig.update_layout(width=300)
    fig.show()

Model 0: Pi = 0.9999999999950286 , kappa = 74.58390652531645


Model 1: Pi = 7.169572391867614e-14 , kappa = 14.79521714840258


Model 2: Pi = 1.3073311197910208e-15 , kappa = 1.7032424065095528e-05


Model 3: Pi = 4.897016225797633e-12 , kappa = 17.707844210187847


Model 4: Pi = 1.2959871063739621e-15 , kappa = 1.703208615468358e-05
